In [51]:
import random
import json

In [52]:
colors = ["red block", "green block", "blue block", "yellow block"]
block_num = [1,2,3,4,5,6,7,8]
str_table = "table"
str_on = "on"

In [54]:
def rand_goal(block_array):
    seed = random.randint(1, 1000000)
    random.seed(seed)
    rand_final = []
    obj_num_goal = random.randint(2, len(block_array))
    randomized_colors = random.sample(block_array, obj_num_goal)
    for i in range(len(randomized_colors) - 1):
        rand_final.append((randomized_colors[i], str_on, randomized_colors[i + 1]))
    rand_final.append((randomized_colors[-1], str_on ,str_table))
    rand_final.reverse()
    return obj_num_goal, seed, rand_final, randomized_colors



def rand_state(block_array, obj_num_goal):
    rand_state_arr = []
    percent_on_blocks = random.randint(1, obj_num_goal)
    for i in range(len(block_array)):
        if i < percent_on_blocks:
            rand_state_arr.append((block_array[i], str_on, str_table))
        else:
            rand_state_arr.append((block_array[i], str_on, block_array[i-1]))
    return rand_state_arr

In [65]:
 def read_data(file_name):
    with open(file_name, "r") as f:
            data = f.read()
            entries = data.split('data_')
            count = 0
            for entry in entries[1:]:
                count+=1
            return count

 def collect_data(num):
     f = open("data_collection", "a")
     count = 0
     offset = read_data(f"data_collection")
     while count < num:
        rand_goal_list = rand_goal(colors)
        rand_state_list = rand_state(rand_goal_list[3], rand_goal_list[0])
        combined_json = {
            "Goal_State": rand_goal_list[2],
            "Current_State": rand_state_list,
            "Pick": None,
            "Place": None
        }

        final_json = json.dumps(combined_json, indent=3)
        f.write(f"data_{count + 1 + offset}:\n")
        f.write(final_json)
        f.write("\n")
        count+=1
     f.close()
collect_data(2)

In [63]:
def update_pick_and_place(file_name):
    with open(file_name, "r") as f:
        data = f.read()
        entries = data.split('data_')

    with open(file_name, "w") as f:
        for entry in entries[1:]:
            lines = entry.split('\n')
            if len(lines) > 1:
                json_data = json.loads('\n'.join(lines[1:]))
                pick = input(f"Enter 'Pick' value for data_{entries.index(entry)}\n goal state: \n {json_data['Goal_State']} \n current state:\n {json_data['Current_State']} ")
                place = input(f"Enter 'Place' value for data_{entries.index(entry)}\n goal state: \n {json_data['Goal_State']} \n current state:\n {json_data['Current_State']} ")
                json_data["Pick"] = pick
                json_data["Place"] = place

                final_json = json.dumps(json_data, indent=3)
                f.write(f"data_{entries.index(entry)}:\n")
                f.write(final_json)
                f.write("\n")

update_pick_and_place("data_collection")

In [66]:
with open("data_collection", "w") as f:
    pass